A 15 to  max 20 pages document. The report should include:

• Specific goals of the work.

• Review of related work.

• Description of your testing procedures.

• Description of the algorithms used. Ensure that the work is reproducible given your
description.

• Results. Ensure to perform a correct experimental analysis (e.g. repeated experiments,
statistical analysis of results).

• Conclusion. Is your line of work worth pursuing? What additional enhancements could be made?

• A short description of the contribution and self-evaluation of each member of the team

• URL address where we can find implementation of algorithms developed and used during the project: 1) source code, together with all data necessary to run the program and a short manual describing how to use/run the program, and 2) a sample run, screenshot, or other indication of system behaviour.

Deadline: 20 June.

# Specific Goals of the Work

- use the teachings on Ensembles (a number of experts who's opinion is combined to increase the probability of it being correct) to estimate the uncertainty within which the predicted value is going to fall

- Die Berechnung der Likelihood of combination of experts being correct vs how it scales with group size

- Why Uncertainty? Agents! See Academic Writing sample



The difference between statistical modelling and machine learning is generally not well defined. Both gain insights form large amounts of data and both utilise complex models in order to make their predictions. 

One of the possible differences is that Statistical Modelling generally depends on the existence of fully specified distributions, while Machine Learning in general is more concerned with point-estimation of target values. 
However, a few approaches seem to be able to combine the benefits of both approaches. These approaches, generally spearheaded by bayesian scientists (who are uncomfortable with point estimations in general, as they always want to know a full distribution over predictions).


- Uncertainty is important for decision making

- See eg. the online bootstrap for explanation why, or Agrawal, S., Com, S., Goyal, N., Mannor, S., Srebro, N., & Williamson, R. C. (2012). Analysis of Thompson Sampling for the Multi-armed Bandit Problem, 2326(39), 1–39. Retrieved from http://proceedings.mlr.press/v23/agrawal12/agrawal12.pdf

- difference between statistical modelling and machine learning ~ uncertainty

- Two approaches:

- Bayesian Approaches to Uncertainty: Fully specified distribution - although most of the time it's approximated because fully bayesian methods are expensive; so what we usually have is either variational bayes or MCMC

- Frequentist Approach to uncertainty:  ‘confidence interval’ (see https://www.bipm.org/cc/CCT/Allowed/26/Disentangling_uncertainty_v14.pdf)

- Ensemble is a method where the prediction of many different models are combined to produce a more elaborate prediction (Find a standard text on ensembles - in the course?)

- Ensembles are very much like asking different experts their opinion in order to find a more likely true prediction (look at the berechnung in one of the assignments)

- Thus theoretically, we can assume that no single expert holds the truth, but we can assume that since different experts have different opinions that are all somewhat grounded in truth, the real model BZW THE BEST POSSIBLE MODEL GIVEN THE DATA AND FORMALISMS we have in place is somewhere betweeen the opinions. Thus we take the predictive mean - the mean of all the predictions - and use that as our final prediction. However, it is possible to extract uncertainty information from this group of experts: If they are all kind of recommending the same thing, it is likely that the optimal recommendation is somewhere close to the value. If all eperts vastly disagree, we can assume that there is not enough information in the Data they have seen, or that the structure of their studies isn't enough, or that there is something wrong otherwise. It is likely that each of them is either basing their decision on wrong data or that they are just randomly guessing (and pretending to know VERY well what they are talking about, as is some experts style).

- This is reflected when we don't only take the mean of the prediction of our Ensemble but also it's standard deviation.

- Thus we can easily infer the uncertainty of our model with almost no extra computaional cost.

- Important: the models need to be sufficiently different in order for this to be a valid method. ASking an expert, his best friend and ten of their studendts might falsely lead to a very confident clique of experts.

- to test for this, use predictive power of one member on the others divided by the predictive power of one ensemble member on the error of the ensemble?


# Review of related work

- Characterizing and Visualizing Predictive Uncertainty in Numerical Ensembles Through Bayesian Model Averaging. (n.d.). Retrieved from http://graphics.cs.ucdavis.edu/~joy/NSF-IIS-1018097/Papers/Vis13_175.pdf

- Adding Uncertainty to Deep Learning – Towards Data Science. (n.d.). Retrieved June 6, 2018, from https://towardsdatascience.com/adding-uncertainty-to-deep-learning-ecc2401f2013

- Renard, B., Kavetski, D., Kuczera, G., Thyer, M., & Franks, S. W. (2010). Understanding predictive uncertainty in hydrologic modeling: The challenge of identifying input and structural errors. Water Resour. Res, 46. http://doi.org/10.1029/2009WR008328

- Kiureghian, A. Der, & Ditlevsen, O. (2009). Aleatory or epistemic? Does it matter? Structural Safety, 31(2), 105–112. http://doi.org/10.1016/J.STRUSAFE.2008.06.020

- Tashman, L. (2000). Out -of-sample tests of forecasting accuracy:ananalysisandreview. International Journal of Forecasting, 16, 437–450. Retrieved from https://www.researchgate.net/profile/Len_Tashman/publication/247087596_Out-of_sample_tests_of_forecasting_accuracy_a_tutorial_and_review/links/5745ceec08ae9f741b430de3.pdf

- Parker, W. S. (2013). Ensemble modeling, uncertainty and robust predictions. Wiley Interdisciplinary Reviews: Climate Change, 4(3), 213–223. http://doi.org/10.1002/wcc.220

- Willink, R., & White, R. (n.d.). Disentangling Classical and Bayesian Approaches to Uncertainty Analysis. Retrieved from https://www.bipm.org/cc/CCT/Allowed/26/Disentangling_uncertainty_v14.pdf

# Description of Testing procedures

- 4 ways of generating uncertainty from an ensemble (Bootstrap, Shuffle Data, Mix of models, Forest)

- Toy dataset

- Boston housing price Dataset

- Cross Validation (Accuracy)

- out of sample distribution prediction (is the uncertainty high for unknowable things)

- converage probability

- Calibration (is the average STD similar to the STD of the data generating model? i.e. (np.std(y) - np.std(y_hay)? where unknown

- correlation between uncertainty and error

- redone 100 times for each experiment



# Results. 
Ensure to perform a correct experimental analysis (e.g. repeated experiments,
statistical analysis of results)



#  Conclusion. 
Is your line of work worth pursuing? What additional enhancements could be made?

- Watch out for more of this

- Deep Learning can be used in this!

- Make sure to have uncorrelated models, otherwise you're deceiving yourself

- Theoretically can combine different kinds of models and can thus be used for other things (deep learning and linear combined results in A little Explainability)


# A short description of the contribution and self-evaluation of each member of the team



# URL address where we can find implementation of algorithms developed and used during the project: 1) source code, together with all data necessary to run the program and a short manual describing how to use/run the program, and 2) a sample run, screenshot, or other indication of system behaviour.
